In [1]:
# !pip install catboost

In [6]:
#импорт библиотек
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier, Pool, cv, MetricVisualizer
from sklearn.metrics import classification_report
import re
import itertools
from google.colab import drive

In [248]:
#подключение google drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
path = '/Users/mikhailgoncharov/moskow-city-hack21/ProjectN5/database/'
path_google_drive = '/content/drive/MyDrive/ProjectN5/'

In [8]:
data = pd.read_csv(path + 'skills_for_prediction_model.csv', usecols = ['id','name','skills'])
# data = pd.read_csv(path_google_drive + 'skills_for_prediction_model.csv', usecols = ['id','name','skills'])

In [9]:
#приводим датасет к необходимому виду
df = pd.get_dummies(data, columns=['skills']).copy()
df.columns = [col[7:] if 'skills_' in col else col for col in df.columns ]

In [10]:
df2 = df.copy()

In [11]:
df.shape,df2.shape

((5357, 720), (5357, 720))

In [12]:
#очищаем текст от символов
def clear_text(text):
    
    text_cleared = re.sub(r"[^a-zA-Zа-яА-ЯёЁ\' ]", ' ', text) #очистка текста
    
    return " ".join(text_cleared.split())  
    

In [13]:
#приводим наименования позиций к общим названиям
def replace_name(raw_series, target_name, exceptions=[]):
  a = set(target_name.split())
  return pd.Series([target_name if val not in exceptions and a == a.intersection(set(clear_text(val).lower().split())) else val for val in raw_series])
  

In [14]:
#составляем лист 
levels = ['junior', 'middle', 'senior']
entire_positions = ['data analyst', 'data scientist', 'data engineer', 'ml engineer']
positions = [ a + " " + b for a,b in list(itertools.product(levels,entire_positions))]
[positions.append(pos) for pos in entire_positions]
positions

['junior data analyst',
 'junior data scientist',
 'junior data engineer',
 'junior ml engineer',
 'middle data analyst',
 'middle data scientist',
 'middle data engineer',
 'middle ml engineer',
 'senior data analyst',
 'senior data scientist',
 'senior data engineer',
 'senior ml engineer',
 'data analyst',
 'data scientist',
 'data engineer',
 'ml engineer']

In [15]:
#заменяем наименования позиций
replaced = df['name']
for pos in positions:
  print(pos, 'has been replaced')
  replaced = replace_name(replaced, pos, positions[:-4])

junior data analyst has been replaced
junior data scientist has been replaced
junior data engineer has been replaced
junior ml engineer has been replaced
middle data analyst has been replaced
middle data scientist has been replaced
middle data engineer has been replaced
middle ml engineer has been replaced
senior data analyst has been replaced
senior data scientist has been replaced
senior data engineer has been replaced
senior ml engineer has been replaced
data analyst has been replaced
data scientist has been replaced
data engineer has been replaced
ml engineer has been replaced


In [16]:
#так же готовим датасет с более усреднёнными позициями (без разбивки на джуна, мида и сеньора)
replaced2 = df['name']
for pos in positions:
  print(pos, 'has been replaced')
  replaced2 = replace_name(replaced, pos)

junior data analyst has been replaced
junior data scientist has been replaced
junior data engineer has been replaced
junior ml engineer has been replaced
middle data analyst has been replaced
middle data scientist has been replaced
middle data engineer has been replaced
middle ml engineer has been replaced
senior data analyst has been replaced
senior data scientist has been replaced
senior data engineer has been replaced
senior ml engineer has been replaced
data analyst has been replaced
data scientist has been replaced
data engineer has been replaced
ml engineer has been replaced


In [17]:
df['name'] = replaced

In [18]:
df2['name'] = replaced2

In [19]:
#удаляем дубликаты
df.drop_duplicates(inplace = True)

In [20]:
df2.drop_duplicates(inplace = True)

In [21]:
#оставляем только вакансии для нашего набора позиций
df_fine = pd.DataFrame(columns=df.columns)
for pos in positions:
  df_fine = df_fine.append(df[df['name'] == pos])


In [22]:
df_fine2 = pd.DataFrame(columns=df.columns)
for pos in entire_positions:
  df_fine2 = df_fine2.append(df[df['name'] == pos])

In [23]:
df_fine.shape,df_fine2.shape

((3779, 720), (3085, 720))

In [24]:
#определяем скилы, не попавшие ни в одну вакансию
zero_cols = []
for col in df_fine.columns:
  if len(df_fine[col].value_counts()/df_fine.shape[0]) == 1:
    zero_cols.append(col)

In [25]:
zero_cols2 = []
for col in df_fine2.columns:
  if len(df_fine2[col].value_counts()/df_fine2.shape[0]) == 1:
    zero_cols2.append(col)

In [26]:
#удаляем их

df_fine.drop(zero_cols, axis=1, inplace=True)

In [27]:
df_fine2.drop(zero_cols2, axis=1, inplace=True)

In [28]:
df_fine2.shape, df_fine.shape, data.shape


((3085, 457), (3779, 527), (5357, 3))

In [29]:
# Модели

In [30]:
target = 'name'

In [31]:
X, y = df_fine.drop(target, axis=1), df_fine[target]
X.drop(['id'], axis=1,inplace=True)


In [32]:
X2, y2 = df_fine2.drop(target, axis=1), df_fine2[target]
X2.drop(['id'], axis=1,inplace=True)

In [33]:
# kf = KFold(n_splits=10)
# kf.get_n_splits(X)

# means, mins, maxs = list(),list(),list()

# for train_index, test_index in kf.split(X):
#     #  print("TRAIN:", train_index, "TEST:", test_index)
#      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
#      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    

#      logit  = LogisticRegression()
#      logit.fit(X_train, y_train)
#      scores = cross_val_score(logit, X_train, y_train, scoring='accuracy')
#      means.append(np.mean(scores))
#      mins.append(min(scores))
#      maxs.append(max(scores))
#     #  y_pred = logit.predict(X_test)

In [34]:
# catboost

In [35]:
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(X, y, test_size=0.2, random_state=1)
X_train_boost, X_val_boost, y_train_boost, y_val_boost = train_test_split(X_train_boost, y_train_boost, test_size=0.25, random_state=1) 
print(f'Train size: {round(X_train_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nVal size:   {round(X_val_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nTest size:  {round(X_test_boost.shape[0] / X.shape[0]*100, 2)}%')


Train size: 59.99%      
Val size:   20.01%      
Test size:  20.01%


In [36]:
train_pool = Pool(
    data=X_train_boost, 
    label=y_train_boost
)

val_pool = Pool(
    data=X_val_boost, 
    label=y_val_boost
)

test_pool = Pool(
    data=X_test_boost, 
    label=y_test_boost
)

In [37]:
model = CatBoostClassifier(
    iterations=3000,
    early_stopping_rounds=1000,
    learning_rate=0.3,
    eval_metric='Accuracy'
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=60
    # train_dir='raw_num_data'
)

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=False,
    plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [40]:
y_pred = model.predict(X_test_boost)
print(classification_report(y_test_boost, y_pred))

                       precision    recall  f1-score   support

         data analyst       0.60      0.58      0.59       278
        data engineer       0.33      0.04      0.07        26
       data scientist       0.51      0.82      0.63       300
  junior data analyst       0.00      0.00      0.00        16
junior data scientist       0.00      0.00      0.00        16
  middle data analyst       0.00      0.00      0.00        12
 middle data engineer       0.00      0.00      0.00         1
middle data scientist       0.00      0.00      0.00        29
          ml engineer       0.00      0.00      0.00         6
  senior data analyst       0.00      0.00      0.00        32
 senior data engineer       0.00      0.00      0.00         2
senior data scientist       0.00      0.00      0.00        38

             accuracy                           0.54       756
            macro avg       0.12      0.12      0.11       756
         weighted avg       0.44      0.54      0.47 

In [285]:
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(X2, y2, test_size=0.2, random_state=1)
X_train_boost, X_val_boost, y_train_boost, y_val_boost = train_test_split(X_train_boost, y_train_boost, test_size=0.25, random_state=1) 
print(f'Train size: {round(X_train_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nVal size:   {round(X_val_boost.shape[0] / X.shape[0]*100 ,2)}%\
      \nTest size:  {round(X_test_boost.shape[0] / X.shape[0]*100, 2)}%')


Train size: 48.98%      
Val size:   16.33%      
Test size:  16.33%


In [41]:
train_pool = Pool(
    data=X_train_boost, 
    label=y_train_boost
)

val_pool = Pool(
    data=X_val_boost, 
    label=y_val_boost
)

test_pool = Pool(
    data=X_test_boost, 
    label=y_test_boost
)

In [48]:
model = CatBoostClassifier(
    iterations=3000,
    early_stopping_rounds=1000,
    learning_rate=0.3,
    eval_metric='Accuracy')
    # snapshot_file='snapshot.bkp',
    # snapshot_interval=60
    # train_dir=path + '/ProjectN5/figures'

model.fit(
    train_pool,
    eval_set=val_pool,
    verbose=False,
    plot=True);

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [49]:
y_pred2 = model.predict(X_test_boost)
print(classification_report(y_test_boost, y_pred2))

                       precision    recall  f1-score   support

         data analyst       0.60      0.58      0.59       278
        data engineer       0.33      0.04      0.07        26
       data scientist       0.51      0.82      0.63       300
  junior data analyst       0.00      0.00      0.00        16
junior data scientist       0.00      0.00      0.00        16
  middle data analyst       0.00      0.00      0.00        12
 middle data engineer       0.00      0.00      0.00         1
middle data scientist       0.00      0.00      0.00        29
          ml engineer       0.00      0.00      0.00         6
  senior data analyst       0.00      0.00      0.00        32
 senior data engineer       0.00      0.00      0.00         2
senior data scientist       0.00      0.00      0.00        38

             accuracy                           0.54       756
            macro avg       0.12      0.12      0.11       756
         weighted avg       0.44      0.54      0.47 

In [50]:
#смотрим, какие предикторы оказались самыми важными
np.array(model.get_feature_importance(
    data=train_pool, 
    type='LossFunctionChange', 
    prettified=True
))

array([['Spark', 0.013312004674195244],
       ['Tableau', 0.0073166859816353735],
       ['Data Analysis', 0.007206091436495088],
       ...,
       ['Проведение опросов', -1.1366720558347865e-05],
       ['Power Query', -4.937666082205361e-05],
       ['Jupyter', -0.0003257176205833101]], dtype=object)

In [51]:
# w = MetricVisualizer('/ML model/')
# w.start()